# Loading the dataset

In [2]:
import pandas
import numpy as np

df = pandas.read_csv('data.csv', index_col='NUM')

In [3]:
del df["NAME"]
del df["COUNTRY"]

In [4]:
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, cross_validate

data, test = train_test_split(
    df,
    test_size = 0.2,
    random_state = 5)

print(data.shape)
print(test.shape)

(41, 6)
(11, 6)


In [5]:
X = data.loc[:,['Mine Annual Production (Million Tonne)',
       'Stripping Ratio', 'Mill Annual Production (Thousand Tonne)',
       'Reserve Mean Grade % Cu EQU.', 'LOM']]
y = data["CAPEX US$ millions"]

test_X = data.loc[:,['Mine Annual Production (Million Tonne)',
       'Stripping Ratio', 'Mill Annual Production (Thousand Tonne)',
       'Reserve Mean Grade % Cu EQU.', 'LOM']]
test_y = data["CAPEX US$ millions"]

In [6]:
num_folds = 6

from sklearn.model_selection import ShuffleSplit

# K-Fold cross validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=6)

# Monte Carlo cross validation
random_splits = ShuffleSplit(n_splits=100, test_size=0.2, random_state=1)

for c in random_splits.split(data):
    print(c)


(array([22, 21, 32, 27, 33, 29, 31, 40,  4, 14, 10, 36, 24, 26, 35, 20, 18,
       25,  6, 13,  7, 39,  1, 16,  0, 15,  5, 11,  9,  8, 12, 37]), array([ 3,  2, 23, 38, 17, 28, 19, 34, 30]))
(array([28, 35, 39, 22, 10, 38, 37,  2, 27, 32,  0, 15, 18,  9, 19, 40, 16,
       20, 26, 12, 11, 17, 24,  4, 36, 21,  6,  3,  7, 30,  8, 13]), array([23, 25,  1, 29, 34,  5, 31, 14, 33]))
(array([30, 26, 11,  2, 16,  5,  4,  6, 20, 31,  3, 13, 17,  1, 37, 24,  0,
       14, 35, 28, 40,  7, 25, 36, 15, 23, 10, 34, 32,  8, 38, 19]), array([27, 22, 18, 12, 21,  9, 33, 29, 39]))
(array([15, 14, 16, 34, 26,  6, 29, 21, 37, 31,  2, 30, 19, 38,  9,  1, 12,
        0, 20, 28, 35, 24, 25, 40, 17, 23, 10, 13, 32, 18,  4,  7]), array([39,  3, 22,  8, 27,  5, 36, 33, 11]))
(array([34,  8, 28,  4, 27,  5, 31,  2,  9, 15, 24,  6, 11, 23, 36, 38, 14,
       17, 29, 19, 39,  0,  7, 21, 25, 32, 10, 20, 33, 13, 37, 22]), array([16, 30, 12, 40, 18,  1,  3, 26, 35]))
(array([39, 15,  9, 26,  4, 25, 37, 32, 13,  8, 10

# Metrics

In [7]:
# Metrics:
#   RMSE -> Root Mean Squared Error
#   R2 -> Coefficient of Determination
#   MAE -> Mean Absolute Error
#   APE -> Absolute Error

In [8]:
from sklearn import metrics

def RMSE(model, X, y):
    return metrics.root_mean_squared_error(y,model.predict(X))

def R2(model, X, y):
    return metrics.r2_score(y,model.predict(X))

def MAE(model, X, y):
    return metrics.mean_absolute_error(y,model.predict(X))

def APE(model, X, y):
    return metrics.mean_absolute_percentage_error(y,model.predict(X))


def all_scores(model, X, y):
    return {
        "RMSE": RMSE(model, X, y),
        "R2": R2(model, X, y),
        "MAE": MAE(model, X, y),
        "APE": APE(model, X, y)
    }



# Models

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from pyswarms.single.global_best import GlobalBestPSO

In [10]:
# average the scores for each model
# then use scores for the optimisation algorithms

In [25]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

# ANN
mlp = MLPRegressor(random_state=1, early_stopping=False)
scores = cross_validate(mlp, X, y, cv=random_splits, scoring=all_scores, return_train_score=True)
print("Training Set")
print("RMSE",np.mean(scores["train_RMSE"]))
print("R2",np.mean(scores["train_R2"]))
print("MAE",np.mean(scores["train_MAE"]))
print("APE",np.mean(scores["train_APE"]))
print("Validation Set")
print("RMSE",np.mean(scores["test_RMSE"]))
print("R2",np.mean(scores["test_R2"]))
print("MAE",np.mean(scores["test_MAE"]))
print("APE",np.mean(scores["test_APE"]))


# Hyperparameters to optimise:
#   -max iterations
#   -number of hidden layers
#   -size of layers
#   -learning rate


# Training Set
# RMSE 1031.1609641480381
# R2 0.5629989516061379
# MAE 710.2267960221794
# APE 0.3683279698501283
# Validation Set
# RMSE 1037.0554540185617
# R2 0.3592855271192601
# MAE 746.6611937168674
# APE 0.37957170166287324


Training Set
RMSE 1093.3271240283605
R2 0.5087378732687482
MAE 723.1863246891122
APE 0.34199746452292373
Validation Set
RMSE 1056.9221753630932
R2 0.34536057281741317
MAE 748.3002189152903
APE 0.3506857983061111


In [19]:
# RF
rf = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_leaf=3)
scores = cross_validate(rf, X, y, cv=random_splits, scoring=all_scores, return_train_score=True)
print(scores.keys())
print("Training Set")
print("RMSE",np.mean(scores["train_RMSE"]))
print("R2",np.mean(scores["train_R2"]))
print("MAE",np.mean(scores["train_MAE"]))
print("APE",np.mean(scores["train_APE"]))
print("Validation Set")
print("RMSE",np.mean(scores["test_RMSE"]))
print("R2",np.mean(scores["test_R2"]))
print("MAE",np.mean(scores["test_MAE"]))
print("APE",np.mean(scores["test_APE"]))

# Hyperparameters to optimise:
#   -number of estimators
#   -max depth
#   -minimum samples per leaf

dict_keys(['fit_time', 'score_time', 'test_RMSE', 'train_RMSE', 'test_R2', 'train_R2', 'test_MAE', 'train_MAE', 'test_APE', 'train_APE'])
Training Set
RMSE 713.5366013491529
R2 0.7874600030144197
MAE 524.3354338093777
APE 0.281212716336767
Validation Set
RMSE 1118.537150434796
R2 0.22283175224100898
MAE 856.8777603844175
APE 0.4558431592958737


In [12]:
# SVM
svr = SVR()

scores = cross_validate(svr, X, y, cv=random_splits, scoring=all_scores, return_train_score=True)
print("Training Set")
print("RMSE",np.mean(scores["train_RMSE"]))
print("R2",np.mean(scores["train_R2"]))
print("MAE",np.mean(scores["train_MAE"]))
print("APE",np.mean(scores["train_APE"]))
print("Validation Set")
print("RMSE",np.mean(scores["test_RMSE"]))
print("R2",np.mean(scores["test_R2"]))
print("MAE",np.mean(scores["test_MAE"]))
print("APE",np.mean(scores["test_APE"]))


Training Set
RMSE 1659.0602490830192
R2 -0.11466168128064899
MAE 1071.456955759421
APE 0.5711130233914902
Validation Set
RMSE 1565.6723646278892
R2 -0.2145234309650804
MAE 1051.0410725264094
APE 0.5490241707548784


In [13]:
# CART Tree (decision tree)
cart = DecisionTreeRegressor()

scores = cross_validate(cart, X, y, cv=random_splits, scoring=all_scores, return_train_score=True)
print("Training Set")
print("RMSE",np.mean(scores["train_RMSE"]))
print("R2",np.mean(scores["train_R2"]))
print("MAE",np.mean(scores["train_MAE"]))
print("APE",np.mean(scores["train_APE"]))
print("Validation Set")
print("RMSE",np.mean(scores["test_RMSE"]))
print("R2",np.mean(scores["test_R2"]))
print("MAE",np.mean(scores["test_MAE"]))
print("APE",np.mean(scores["test_APE"]))

print("Hyperparameters:", cart.get_params())

Training Set
RMSE 0.0
R2 1.0
MAE 0.0
APE 0.0
Validation Set
RMSE 1357.8027204072475
R2 -0.0702878173163862
MAE 1000.7377777777779
APE 0.5457732737140774
Hyperparameters: {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}


# Hyperparameter Optimisation

In [14]:
def testANN(param):
    # max iterations
    # size of layers
    # number of hidden layers
    # learning rate (Scale??)
    mlp = MLPRegressor(max_iter=round(param[0]), hidden_layer_sizes=[round(param[1]) for _ in range(round(param[2]))], learning_rate_init=param[3],random_state=1, early_stopping=False)
    scores = cross_validate(mlp, X, y, cv=random_splits, scoring=all_scores, return_train_score=True)
    return np.mean(scores["test_APE"])-np.mean(scores["test_R2"])

def testRF(param):
    # number of estimators
    # max depth
    # minimum samples per leaf
    rf = RandomForestRegressor(n_estimators=round(param[0]), max_depth=round(param[1]), min_samples_leaf=round(param[2]))
    scores = cross_validate(rf, X, y, cv=random_splits, scoring=all_scores, return_train_score=True)
    return np.mean(scores["test_APE"])-np.mean(scores["test_R2"])

def testSVR(param):
    # C
    # epsilon?
    svr = SVR(C=param[0], epsilon=param[1])
    scores = cross_validate(svr, X, y, cv=random_splits, scoring=all_scores, return_train_score=True)
    return np.mean(scores["test_APE"])-np.mean(scores["test_R2"])

def testDT(param):#CART Tree
    # max depth
    # minimum samples per leaf
    dt = DecisionTreeRegressor(max_depth=round(param[0]), min_samples_leaf=round(param[1]))
    scores = cross_validate(dt, X, y, cv=random_splits, scoring=all_scores, return_train_score=True)
    return np.mean(scores["test_APE"])-np.mean(scores["test_R2"])


def optimiseModel(params, modelfunc=testRF):
    out = np.array([modelfunc(param) for param in params])
    return out

In [15]:

bounds = (np.array([10,  2,  1]),
          np.array([100, 15, 10]))
# bounds:
#   upper + lower bounds of parameter values to test
#   format -> (np array of upper bounds,
#              np array of lower bounds)

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}# i think just keep these the same
optimizer = GlobalBestPSO(n_particles=5, dimensions=3, options=options, bounds=bounds)# dimensions = number of parameters to optimise

cost, pos = optimizer.optimize(optimiseModel, 10, verbose=True, modelfunc=testRF) #change this to optimise different models (e.g. testRF, testSVR, testDT)

print(cost)
print("optimal parameters: ",pos)




# --------------------- RF experimentation results ----------------------

# R2 (10 particles, 100 iter)
# 59   6   3
# 57  11   3

# R2 (10 particles, 1000 iter)
# 42  11   3

# R2+APE (10 particles, 500 iter)
# 49  11   2
# 45   9   4

# R2+APE (20 particles, 500 iter)
# 84   9   3
# 61  10   3

# R2+APE (20 particles, 100 iter)
# 84   9   3
# 60  12   2

# -> increased MC crossvalidation n_splits to increase consistency

# -----------------------------------------------------------------------




# could use multiprocessing to speed it up, but then can't use ipynb

2024-05-01 19:57:23,084 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|10/10, best_cost=0.096
2024-05-01 20:00:43,077 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.09596891661081008, best pos: [90.63889129 13.83597348  3.98377291]


0.09596891661081008
optimal parameters:  [90.63889129 13.83597348  3.98377291]


In [16]:
np.save("pos_history",np.array(optimizer.pos_history))

In [38]:
# NEXT: try plotting optimisation
from pyswarms.utils.plotters.plotters import plot_contour
from pyswarms.utils.plotters.formatters import Designer
import matplotlib.animation as animation
from matplotlib import pyplot as plt
from IPython.display import Image

print(len(optimizer.pos_history))
print(optimizer.pos_history[0].shape)

print(optimizer.pos_history[0][:5])

pos_history = []
for pos_list in optimizer.pos_history:
    pos_history.append(np.delete(pos_list,1,1))


print(len(pos_history))
print(pos_history[0].shape)
print(pos_history[0][:5])

#needs to be 2d array
anim = plot_contour(pos_history, designer=Designer(limits=[(10,100),(1,10)]))

anim.save('plot2.gif', writer='imagemagick', fps=10)


100
(20, 3)
[[67.51098057 11.64340192  3.58862353]
 [78.16407114  9.79862685  9.30253853]
 [51.16971792  6.16177102  3.44817495]
 [95.24126042  7.79798048  6.16330602]
 [21.6481137   2.4758367   4.85837461]]
100
(20, 2)
[[67.51098057  3.58862353]
 [78.16407114  9.30253853]
 [51.16971792  3.44817495]
 [95.24126042  6.16330602]
 [21.6481137   4.85837461]]


<IPython.core.display.Javascript object>

2024-05-01 00:40:39,612 - matplotlib.animation - INFO - Animation.save using <class 'matplotlib.animation.ImageMagickWriter'>
2024-05-01 00:40:39,613 - matplotlib.animation - INFO - MovieWriter._run: running command: 'C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe' -size 1000x800 -depth 8 -delay 10.0 -loop 0 rgba:- -layers OptimizePlus plot2.gif


# Testing

In [24]:
#use optimal hyperparameters to train model, then test
mlp = MLPRegressor(max_iter=400, hidden_layer_sizes=(20), learning_rate_init=0.060, random_state=1, early_stopping=False)
mlp.fit(X,y)
print(RMSE(mlp,test_X,test_y))
print(R2(mlp,test_X,test_y))
print(MAE(mlp,test_X,test_y))
print(APE(mlp,test_X,test_y))

#mlp.score(test_X,test_y)

1051.3603525614853
0.5559267577322748
712.8431726991315
0.3642219775069245


To add later on if time:  
-preprocessing (outliers, etc.)  
-ensemble methods (ANN + SVM)